In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#
import os
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random
from tqdm import tqdm

# --- Feature Engineering Functions ---
def remove_gravity_from_acc(acc_data, rot_data):
    """Remove gravity component from accelerometer data"""
    if isinstance(acc_data, pd.DataFrame):
        acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    else:
        acc_values = acc_data

    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    gravity_world = np.array([0, 0, 9.802])

    for i in range(num_samples):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :]
            continue

        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
            linear_accel[i, :] = acc_values[i, :]

    return linear_accel.astype(np.float32)

def calculate_angular_velocity_from_quat(rot_data, time_delta=1/200):
    """Calculate angular velocity from quaternion derivatives"""
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_vel = np.zeros((num_samples, 3))

    for i in range(num_samples - 1):
        q_t = quat_values[i]
        q_t_plus_dt = quat_values[i+1]

        if np.all(np.isnan(q_t)) or np.all(np.isclose(q_t, 0)) or \
           np.all(np.isnan(q_t_plus_dt)) or np.all(np.isclose(q_t_plus_dt, 0)):
            continue

        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)
            delta_rot = rot_t.inv() * rot_t_plus_dt
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError:
            pass

    return angular_vel.astype(np.float32)

def calculate_angular_distance(rot_data):
    """Calculate angular distance between successive quaternions"""
    if isinstance(rot_data, pd.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_dist = np.zeros(num_samples)

    for i in range(num_samples - 1):
        q1 = quat_values[i]
        q2 = quat_values[i+1]

        if np.all(np.isnan(q1)) or np.all(np.isclose(q1, 0)) or \
           np.all(np.isnan(q2)) or np.all(np.isclose(q2, 0)):
            angular_dist[i] = 0
            continue

        try:
            r1 = R.from_quat(q1)
            r2 = R.from_quat(q2)
            relative_rotation = r1.inv() * r2
            angle = np.linalg.norm(relative_rotation.as_rotvec())
            angular_dist[i] = angle
        except ValueError:
            angular_dist[i] = 0

    return angular_dist.astype(np.float32)

# --- Simple Improved Model ---
class ImprovedCNNIMUModel(nn.Module):
    """Improved CNN with GELU activation - often better than ReLU for deep networks"""
    def __init__(self, input_dim, num_classes, dropout_rate=0.25):
        super().__init__()

        # Better feature MLP with progressive dropout
        self.feature_mlp = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),  # GELU activation
            nn.Dropout(dropout_rate * 0.4),

            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),  # GELU activation
            nn.Dropout(dropout_rate * 0.6),

            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.GELU()  # GELU activation
        )

        # Improved temporal convolutions with 1D dropout
        self.temporal_conv = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.GELU(),  # GELU activation
            nn.Dropout1d(dropout_rate * 0.3),

            nn.Conv1d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.GELU(),  # GELU activation
            nn.Dropout1d(dropout_rate * 0.5)
        )

        # Multi-scale feature extraction
        self.multi_scale = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(256, 64, kernel_size=3, padding=1),
                nn.BatchNorm1d(64),
                nn.GELU()  # GELU activation
            ),
            nn.Sequential(
                nn.Conv1d(256, 64, kernel_size=5, padding=2),
                nn.BatchNorm1d(64),
                nn.GELU()  # GELU activation
            ),
            nn.Sequential(
                nn.Conv1d(256, 64, kernel_size=7, padding=3),
                nn.BatchNorm1d(64),
                nn.GELU()  # GELU activation
            ),
            nn.Sequential(
                nn.Conv1d(256, 64, kernel_size=11, padding=5),
                nn.BatchNorm1d(64),
                nn.GELU()  # GELU activation
            )
        ])

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv1d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm1d(512),
            nn.GELU(),  # GELU activation
            nn.Dropout1d(dropout_rate * 0.5)
        )

        # Enhanced pyramid pooling
        self.pyramid_pool = nn.ModuleList([
            nn.AdaptiveAvgPool1d(1),
            nn.AdaptiveAvgPool1d(2),
            nn.AdaptiveAvgPool1d(4),
            nn.AdaptiveMaxPool1d(1)
        ])

        # Improved classifier
        self.classifier = nn.Sequential(
            nn.Linear(512 * 8, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),  # GELU activation
            nn.Dropout(dropout_rate),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),  # GELU activation
            nn.Dropout(dropout_rate * 0.8),

            nn.Linear(256, num_classes)
        )

    def forward(self, x, lengths):
        batch_size, seq_len, feat_dim = x.shape

        # Feature transformation
        x_reshaped = x.view(-1, feat_dim)
        x_transformed = self.feature_mlp(x_reshaped)
        x = x_transformed.view(batch_size, seq_len, -1).transpose(1, 2)

        # Temporal feature extraction
        x = self.temporal_conv(x)

        # Multi-scale temporal feature extraction
        scale_features = []
        for conv in self.multi_scale:
            scale_features.append(conv(x))
        x = torch.cat(scale_features, dim=1)

        # Bottleneck
        x = self.bottleneck(x)

        # Masking for variable length sequences
        max_time = x.size(2)
        mask = torch.arange(max_time, device=x.device)[None, :] < lengths[:, None]
        mask = mask.unsqueeze(1).float()
        x = x * mask

        # Enhanced pyramid pooling
        pyramid_features = []
        for pool in self.pyramid_pool:
            pooled = pool(x)
            pooled = pooled.view(pooled.size(0), -1)
            pyramid_features.append(pooled)

        # Combine features
        features = torch.cat(pyramid_features, dim=1)

        # Classification
        return self.classifier(features)

# --- Data Augmentation (Original) ---
class Augmenter:
    def __init__(self, scale_range=(0.9, 1.1), noise_std=0.02, flip_prob=0.5):
        self.scale_range = scale_range
        self.noise_std = noise_std
        self.flip_prob = flip_prob

    def apply_augmentations(self, data):
        data = data.astype(np.float32)

        if self.scale_range is not None:
            scale = np.random.uniform(self.scale_range[0], self.scale_range[1])
            data = data * scale

        if self.noise_std is not None:
            noise = np.random.normal(0, self.noise_std, data.shape).astype(np.float32)
            data = data + noise

        if self.flip_prob is not None:
            if np.random.rand() < self.flip_prob:
                data = np.flip(data, axis=0).copy()

        return data.astype(np.float32)

class IMUDataset(Dataset):
    def __init__(self, df, feat_cols, label_map, augment=False):
        self.samples = []
        self.feat_cols = feat_cols
        self.label_map = label_map
        self.augment = augment
        self.augmenter = Augmenter() if augment else None

        for sid, group in df.groupby('sequence_id'):
            arr = group[feat_cols].values.astype(np.float32)
            length = arr.shape[0]
            gesture = group['gesture'].iloc[0]

            self.samples.append({
                'data': arr,
                'length': length,
                'label': label_map[gesture],
                'gesture': gesture,
                'sequence_id': sid
            })

        if augment:
            self.gesture_map = {}
            for sample in self.samples:
                gesture = sample['gesture']
                self.gesture_map.setdefault(gesture, []).append(sample)

    def __len__(self):
        if not self.augment:
            return len(self.samples)
        return int(len(self.samples) * 3)

    def __getitem__(self, idx):
        n = len(self.samples)

        if not self.augment:
            sample = self.samples[idx]
            return (
                torch.from_numpy(sample['data']),
                sample['length'],
                sample['label']
            )

        base_idx = idx % n
        sample_type = idx // n

        if sample_type == 0:
            sample = self.samples[base_idx]
            data = sample['data'].copy()
            if self.augmenter:
                data = self.augmenter.apply_augmentations(data)
            return (
                torch.from_numpy(data),
                sample['length'],
                sample['label']
            )

        elif sample_type == 1:
            base_sample = self.samples[base_idx]
            gesture = base_sample['gesture']
            candidates = self.gesture_map.get(gesture, [])

            if len(candidates) < 4:
                data = base_sample['data'].copy()
                if self.augmenter:
                    data = self.augmenter.apply_augmentations(data)
                return (
                    torch.from_numpy(data),
                    base_sample['length'],
                    base_sample['label']
                )

            partner = random.choice(candidates)
            while partner['sequence_id'] == base_sample['sequence_id']:
                partner = random.choice(candidates)

            lam = np.random.beta(0.4, 0.4)
            min_len = min(base_sample['length'], partner['length'])
            base_data = base_sample['data'][:min_len].astype(np.float32)
            part_data = partner['data'][:min_len].astype(np.float32)
            mixed = (lam * base_data + (1 - lam) * part_data).astype(np.float32)

            return (
                torch.from_numpy(mixed),
                min_len,
                base_sample['label']
            )

        else:
            sample = self.samples[base_idx]
            data = sample['data'].copy()
            data = self.augmenter.apply_augmentations(data)
            return (
                torch.from_numpy(data),
                sample['length'],
                sample['label']
            )

def collate_fn(batch):
    feats, lens, lbls = zip(*batch)
    lens = torch.tensor(lens, dtype=torch.long)
    lbls = torch.tensor(lbls, dtype=torch.long)
    padded = nn.utils.rnn.pad_sequence(feats, batch_first=True)
    return padded, lens, lbls

# --- Training Functions ---
def train_one_epoch_improved(model, loader, optimizer, device):
    """Improved training function with label smoothing"""
    model.train()
    total_loss = 0.0

    # Label smoothing for better generalization
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    for x, lengths, y in tqdm(loader, desc="Training"):
        x = x.float().to(device)
        lengths = lengths.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        out = model(x, lengths)
        loss = criterion(out, y)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item() * x.size(0)

    return total_loss / len(loader.dataset)

def evaluate_improved(model, loader, device, non_target_idxs):
    """Same evaluation function"""
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for x, lengths, y in loader:
            x = x.float().to(device)
            lengths = lengths.to(device)
            out = model(x, lengths)
            p = out.argmax(1).cpu().numpy()
            preds.append(p)
            trues.append(y.numpy())

    preds = np.concatenate(preds)
    trues = np.concatenate(trues)

    # Binary F1
    true_bin = ~np.isin(trues, non_target_idxs)
    pred_bin = ~np.isin(preds, non_target_idxs)
    binary_f1 = f1_score(true_bin, pred_bin)

    # Macro F1
    new_non_target_label = np.max(trues) + 1
    trues_collapsed = np.where(np.isin(trues, non_target_idxs), new_non_target_label, trues)
    preds_collapsed = np.where(np.isin(preds, non_target_idxs), new_non_target_label, preds)

    macro_f1 = f1_score(trues_collapsed, preds_collapsed, average='macro',
                       labels=np.unique(trues_collapsed))

    return binary_f1, macro_f1

In [ ]:

    # Load data
train_csv = '/content/drive/MyDrive/cmi-detect-behavior-with-sensor-data/train.csv'
print("Loading and cleaning data...")
df = pd.read_csv(train_csv)
df.sort_values(['sequence_id', 'sequence_counter'], inplace=True)

# Replace NaN values with 0 in rotation columns
rotation_cols = ['rot_x', 'rot_y', 'rot_z', 'rot_w']
df[rotation_cols] = df[rotation_cols].fillna(0)

# Define initial IMU columns
initial_imu_cols = ['acc_x', 'acc_y', 'acc_z', 'rot_x', 'rot_y', 'rot_z', 'rot_w']

# --- Feature Engineering ---
print("Calculating engineered features...")

# Base features
df['acc_mag'] = np.sqrt(df['acc_x']**2 + df['acc_y']**2 + df['acc_z']**2).astype(np.float32)
df['rot_angle'] = (2 * np.arccos(df['rot_w'].clip(-1, 1))).astype(np.float32)

# Derivatives
df['acc_mag_jerk'] = df.groupby('sequence_id')['acc_mag'].diff().fillna(0).astype(np.float32)
df['rot_angle_vel'] = df.groupby('sequence_id')['rot_angle'].diff().fillna(0).astype(np.float32)

# Second derivatives
df['acc_x_jerk'] = df.groupby('sequence_id')['acc_x'].diff().fillna(0).astype(np.float32)
df['acc_y_jerk'] = df.groupby('sequence_id')['acc_y'].diff().fillna(0).astype(np.float32)
df['acc_z_jerk'] = df.groupby('sequence_id')['acc_z'].diff().fillna(0).astype(np.float32)
df['acc_x_jerk_jerk'] = df.groupby('sequence_id')['acc_x_jerk'].diff().fillna(0).astype(np.float32)
df['acc_y_jerk_jerk'] = df.groupby('sequence_id')['acc_y_jerk'].diff().fillna(0).astype(np.float32)
df['acc_z_jerk_jerk'] = df.groupby('sequence_id')['acc_z_jerk'].diff().fillna(0).astype(np.float32)

# Gravity-removed acceleration
print("  Removing gravity...")
linear_accel_list = []
for seq_id, group in df.groupby('sequence_id'):
    acc_data = group[['acc_x', 'acc_y', 'acc_z']]
    rot_data = group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
    linear_accel = remove_gravity_from_acc(acc_data, rot_data)
    linear_accel_list.append(
        pd.DataFrame(linear_accel, columns=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'], index=group.index)
    )
df = pd.concat([df, pd.concat(linear_accel_list)], axis=1)

# Linear acceleration features
df['linear_acc_mag'] = np.sqrt(df['linear_acc_x']**2 + df['linear_acc_y']**2 + df['linear_acc_z']**2).astype(np.float32)
df['linear_acc_mag_jerk'] = df.groupby('sequence_id')['linear_acc_mag'].diff().fillna(0).astype(np.float32)

# Angular velocity
print("  Calculating angular velocity...")
angular_vel_list = []
for seq_id, group in df.groupby('sequence_id'):
    rot_data = group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
    angular_vel = calculate_angular_velocity_from_quat(rot_data)
    angular_vel_list.append(
        pd.DataFrame(angular_vel, columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'], index=group.index)
    )
df = pd.concat([df, pd.concat(angular_vel_list)], axis=1)

# Angular velocity features
df['angular_vel_mag'] = np.sqrt(df['angular_vel_x']**2 + df['angular_vel_y']**2 + df['angular_vel_z']**2).astype(np.float32)
df['angular_vel_mag_jerk'] = df.groupby('sequence_id')['angular_vel_mag'].diff().fillna(0).astype(np.float32)

# Angular distance
print("  Calculating angular distance...")
angular_dist_list = []
for seq_id, group in df.groupby('sequence_id'):
    rot_data = group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]
    angular_dist = calculate_angular_distance(rot_data)
    angular_dist_list.append(
        pd.DataFrame(angular_dist, columns=['angular_distance'], index=group.index)
    )
df = pd.concat([df, pd.concat(angular_dist_list)], axis=1)

# Ensure all numeric columns are float32
for col in df.select_dtypes(include=[np.number]).columns:
    df[col] = df[col].astype(np.float32)

# Define all IMU columns #
imu_cols = initial_imu_cols + [
    'acc_mag', 'rot_angle', 'acc_mag_jerk', 'rot_angle_vel',
    'acc_x_jerk', 'acc_y_jerk', 'acc_z_jerk',
    'acc_x_jerk_jerk', 'acc_y_jerk_jerk', 'acc_z_jerk_jerk',
    # 'linear_acc_x', 'linear_acc_y', 'linear_acc_z', 'linear_acc_mag', 'linear_acc_mag_jerk',
    'angular_vel_x', 'angular_vel_y', 'angular_vel_z', 'angular_vel_mag', 'angular_vel_mag_jerk',
    'angular_distance'
]

print(f"Using {len(imu_cols)} IMU features")

# --- Labels & grouping info ---
gestures = sorted(df['gesture'].unique())
label_map = {g: i for i, g in enumerate(gestures)}
non_target_list = [
    'Drink from bottle/cup','Glasses on/off','Pull air toward your face',
    'Pinch knee/leg skin','Scratch knee/leg skin','Write name on leg',
    'Text on phone','Feel around in tray and pull out an object',
    'Write name in air','Wave hello'
]
non_target_idxs = [label_map[g] for g in non_target_list if g in label_map]

# Sequence info
seq_info = df.groupby('sequence_id')\
              .agg(gesture=('gesture','first'), subject=('subject','first'))\
              .reset_index()

# --- Training Configuration ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Hyperparameters
EPOCHS = 100
BATCH_SIZE = 32
LR = 0.001
WEIGHT_DECAY = 1e-4
K_FOLDS = 5

# Cross-validation
sgkf = StratifiedGroupKFold(n_splits=K_FOLDS, shuffle=True, random_state=10086)
fold_results = []
# --- Main Training Script ---
def main():
    print("=== Simple Improved CNN IMU Model Training ===")



    for fold, (train_idx, val_idx) in enumerate(
        sgkf.split(seq_info['sequence_id'], seq_info['gesture'], groups=seq_info['subject'])
    ):
        print(f"\n=== Fold {fold+1}/{K_FOLDS} ===")

        # Get sequence IDs for train/val
        train_seq_ids = seq_info.iloc[train_idx]['sequence_id'].values
        val_seq_ids = seq_info.iloc[val_idx]['sequence_id'].values

        # Filter dataframe
        train_df = df[df['sequence_id'].isin(train_seq_ids)]
        val_df = df[df['sequence_id'].isin(val_seq_ids)]

        # Create datasets and dataloaders
        train_dataset = IMUDataset(train_df, imu_cols, label_map, augment=True)
        val_dataset = IMUDataset(val_df, imu_cols, label_map, augment=False)

        train_loader = DataLoader(
            train_dataset,
            batch_size=BATCH_SIZE,
            shuffle=True,
            collate_fn=collate_fn,
            pin_memory=True,
            num_workers=2
        )
        val_loader = DataLoader(
            val_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            collate_fn=collate_fn,
            pin_memory=True
        )

        # Initialize simple improved model
        model = ImprovedCNNIMUModel(len(imu_cols), len(gestures), dropout_rate=0.25).to(device)

        # Optimizer and scheduler (no warmup)
        optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=LR,
            epochs=EPOCHS,
            steps_per_epoch=len(train_loader),
            anneal_strategy='cos'
        )

        # Training loop
        best_score = 0.0
        best_binary_f1 = 0.0
        best_macro_f1 = 0.0
        patience = 15
        patience_counter = 0

        for epoch in range(EPOCHS):
            # Train
            train_loss = train_one_epoch_improved(model, train_loader, optimizer, device)
            scheduler.step()

            # Evaluate
            binary_f1, macro_f1 = evaluate_improved(model, val_loader, device, non_target_idxs)
            combined_score = (binary_f1 + macro_f1) / 2

            print(f"Epoch {epoch+1}/{EPOCHS}: "
                  f"Train Loss: {train_loss:.4f}, "
                  f"Val Binary F1: {binary_f1:.4f}, "
                  f"Val Macro F1: {macro_f1:.4f}, "
                  f"Combined Score: {combined_score:.4f}")

            # Early stopping and best model saving based on combined score
            if combined_score > best_score:
                best_score = combined_score
                best_binary_f1 = binary_f1
                best_macro_f1 = macro_f1
                patience_counter = 0
                torch.save(model.state_dict(), f"best_improved_model_fold{fold}.pth")
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        fold_results.append({
            'fold': fold,
            'best_score': best_score,
            'best_binary_f1': best_binary_f1,
            'best_macro_f1': best_macro_f1
        })
        print(f"Fold {fold+1} Best Score: {best_score:.4f} (Binary: {best_binary_f1:.4f}, Macro: {best_macro_f1:.4f})")

    # Calculate and print final results
    best_scores = [res['best_score'] for res in fold_results]
    best_binary_f1s = [res['best_binary_f1'] for res in fold_results]
    best_macro_f1s = [res['best_macro_f1'] for res in fold_results]

    mean_best_score = np.mean(best_scores)
    std_best_score = np.std(best_scores)
    mean_binary_f1 = np.mean(best_binary_f1s)
    mean_macro_f1 = np.mean(best_macro_f1s)

    print(f"\n=== Simple Improved Model Cross-Validation Results ===")
    print(f"Average Best Score (Binary+Macro)/2: {mean_best_score:.4f} ± {std_best_score:.4f}")
    print(f"Average Best Binary F1: {mean_binary_f1:.4f}")
    print(f"Average Best Macro F1: {mean_macro_f1:.4f}")
    print(f"Individual Best Scores: {[f'{score:.4f}' for score in best_scores]}")
    print(f"Individual Binary F1s: {[f'{score:.4f}' for score in best_binary_f1s]}")
    print(f"Individual Macro F1s: {[f'{score:.4f}' for score in best_macro_f1s]}")

if __name__ == "__main__":
    main()

Loading and cleaning data...
Calculating engineered features...
  Removing gravity...
  Calculating angular velocity...
  Calculating angular distance...
Using 23 IMU features
Using device: cuda
=== Simple Improved CNN IMU Model Training ===

=== Fold 1/5 ===


Training: 100%|██████████| 612/612 [00:07<00:00, 82.87it/s]


Epoch 1/100: Train Loss: 2.5477, Val Binary F1: 0.8856, Val Macro F1: 0.2260, Combined Score: 0.5558


Training: 100%|██████████| 612/612 [00:07<00:00, 82.52it/s]


Epoch 2/100: Train Loss: 2.1011, Val Binary F1: 0.9129, Val Macro F1: 0.3088, Combined Score: 0.6108


...

Epoch 47/100: Train Loss: 0.8410, Val Binary F1: 0.9743, Val Macro F1: 0.5689, Combined Score: 0.7716


Training: 100%|██████████| 607/607 [00:07<00:00, 81.23it/s]


Epoch 48/100: Train Loss: 0.8419, Val Binary F1: 0.9754, Val Macro F1: 0.5914, Combined Score: 0.7834


Training: 100%|██████████| 607/607 [00:07<00:00, 81.09it/s]


Epoch 49/100: Train Loss: 0.8325, Val Binary F1: 0.9792, Val Macro F1: 0.5776, Combined Score: 0.7784
Early stopping at epoch 49
Fold 5 Best Score: 0.7862 (Binary: 0.9788, Macro: 0.5935)

=== Simple Improved Model Cross-Validation Results ===
Average Best Score (Binary+Macro)/2: 0.7817 ± 0.0111
Average Best Binary F1: 0.9772
Average Best Macro F1: 0.5862
Individual Best Scores: ['0.7622', '0.7934', '0.7775', '0.7893', '0.7862']
Individual Binary F1s: ['0.9654', '0.9799', '0.9772', '0.9848', '0.9788']
Individual Macro F1s: ['0.5591', '0.6070', '0.5777', '0.5937', '0.5935']
